In [1]:
import os; os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import random
import torch
import numpy as np


def setup_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")
    
SEED = 1000
setup_reproducibility(SEED)

In [2]:
import os
from collections import OrderedDict
from transformers import get_cosine_schedule_with_warmup
from scipy import signal
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from huggingface_hub import login, snapshot_download
from tqdm.auto import tqdm


def average_state_dicts(state_dict_list):
    n = len(state_dict_list)
    # Ensure we don't modify the originals
    avg_sd = OrderedDict()

    # Iterate over every parameter/buffer key
    for k in state_dict_list[0]:
        # sum across models → float32 to avoid overflow on int types
        avg = sum(sd[k].float() for sd in state_dict_list) / n
        # cast back to original dtype if needed
        avg_sd[k] = avg.to(dtype=state_dict_list[0][k].dtype)

    return avg_sd


def get_ckpt_paths(output_dir, keyword):
    output_files = sorted(os.listdir(output_dir))

    ckpt_paths = []
    for f in output_files:
        if keyword in f and "csv" not in f:
            ckpt_path = os.path.join(output_dir, f)
            ckpt_paths.append(ckpt_path)
            ckpt = torch.load(ckpt_path, weights_only=False)
            print(ckpt_path, ckpt["epoch"], ckpt["score"])
            
    return ckpt_paths


def rest(t=4000):
    import time
    [time.sleep(1) for i in range(t)]
        
        
def generate_csv(preds, name):
    column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
    preds_df = pd.DataFrame(preds, columns=column_names)
    preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
    preds_df.to_csv(name, index=False)
    
    
def get_ckpt(path):
    return torch.load(path, weights_only=False)


def cuda_to_np(tensor):
    return tensor.cpu().detach().numpy()


def get_scheduler(optimizer, train_dl, epochs):
    total_training_steps = len(train_dl) * epochs
    warmup_steps = int(total_training_steps * 0.05)  # e.g. 5% warmup
    
    return get_cosine_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_training_steps
    )


def get_stats(tensor, p=True, r=False, minmax=False):
    if minmax:
        min, max = tensor.min(), tensor.max()
        mean, std = tensor.mean(), tensor.std()
        if p: print(f"Min: {min}, Max: {max}, Mean: {mean}, Std: {std}")
        if r: return min, max, mean, std
    else:
        mean, std = tensor.mean(), tensor.std()
        if p: print(f"Mean: {mean}, Std: {std}")
        if r: return mean, std
    
    
def zscore(tensor, mean=None, std=None):
    if mean is None: mean = tensor.mean()
    if std is None: std = tensor.std()
    return (tensor - mean) / (std + 1e-8)


def reverse_zscore(tensor, mu, sigma):
    return (tensor * sigma) + mu


def get_model_size(model):
    print(sum(p.numel() for p in model.parameters()) / 1e6)
    

def get_index(iterable):
    return random.randint(0, len(iterable) - 1)


def get_indices(iterable, n):
    return random.sample(range(len(iterable)), n)


def split(inputs, targets, seed):
    return train_test_split(
        inputs,
        targets, 
        test_size=0.2,
        shuffle=True, 
        random_state=seed
    ) 


def show_waves(waves, dpi=100):
    """
    waves: numpy array of shape (3, N)
    Creates three separate figures that stretch wide.
    """
    N = waves.shape[1]
    t = np.arange(N)

    # Wide aspect ratio; height modest so each window fills width
    for i in range(waves.shape[0]):
        fig = plt.figure(figsize=(14, 4), dpi=dpi)  # wide figure
        ax = fig.add_subplot(111)
        ax.plot(t, waves[i], linewidth=1)
        ax.set_title(f"Wave {i+1}")
        ax.set_xlabel("Sample")
        ax.set_ylabel("Amplitude")
        ax.grid(True)
        fig.tight_layout()  # reduce margins to use width
        
    plt.show()
    
    
def hf_ds_download(hf_token, repo_id):
    login(hf_token[1:])
    return snapshot_download(repo_id, repo_type="dataset")


def get_spectra_features(X, b=False):
    """Create multi-channel features from spectra: raw, 1st derivative, 2nd derivative."""
    X_processed = np.zeros_like(X)
    # Baseline correction and SNV
    for i in tqdm(range(X.shape[0])):
        poly = np.polyfit(np.arange(X.shape[1]), X[i], 3)
        baseline = np.polyval(poly, np.arange(X.shape[1]))
        corrected_spec = X[i] - baseline
        #X_processed[i] = (corrected_spec - corrected_spec.mean()) / (corrected_spec.std() + 1e-8)
        X_processed[i] = corrected_spec
        
    # Calculate derivatives
    deriv1 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=1, axis=1)
    deriv2 = signal.savgol_filter(X_processed, window_length=11, polyorder=3, deriv=2, axis=1)

    if b: return np.stack([X_processed, deriv1, deriv2], axis=1)
    return np.stack([deriv1, deriv2], axis=1)

In [3]:
import os

if True:
    path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
    files = os.listdir(path)
    [print((i, files[i])) for i in range(len(files))]

(0, 'sample_submission.csv')
(1, 'timegate.csv')
(2, 'mettler_toledo.csv')
(3, 'kaiser.csv')
(4, 'anton_532.csv')
(5, 'transfer_plate.csv')
(6, '96_samples.csv')
(7, 'tornado.csv')
(8, 'tec5.csv')
(9, 'metrohm.csv')
(10, 'anton_785.csv')


In [4]:
if False:
    hf_token = "xhf_XURkoNhwOIPtEdHfNeRpVkjEwKSkhtigFi"
    path = hf_ds_download(hf_token, "ArbaazBeg/kaggle-spectogram")
    files = os.listdir(path)
    [(i, files[i]) for i in range(len(files))]

In [5]:
import pandas as pd


def load_comp_data(filepath, is_train=True):
    """Load and preprocess the Raman spectroscopy data"""
    if is_train:
        df = pd.read_csv(filepath)
        # Extract target variables
        target_cols = ['Glucose (g/L)', 'Sodium Acetate (g/L)', 'Magnesium Acetate (g/L)']
        y = df[target_cols].dropna().values
        
        # Process spectral data
        X = df.iloc[:, :-4] # Remove last 4 columns (analyte info and targets)
    else:
        df = pd.read_csv(filepath, header=None)
        X = df
        y = None
    
    # Set column names
    X.columns = ["sample_id"] + [str(i) for i in range(X.shape[1]-1)]
    
    # Fill sample_id using forward fill
    X['sample_id'] = X['sample_id'].ffill()
    
    # Clean sample_id
    if is_train:
        X['sample_id'] = X['sample_id'].str.strip()
    else:
        X['sample_id'] = X['sample_id'].str.strip().str.replace('sample', '').astype(int)
    
    # Clean spectral data (remove brackets)
    spectral_cols = X.columns[1:]
    for col in spectral_cols:
        X[col] = X[col].astype(str).str.replace('[', '', regex=False).str.replace(']', '', regex=False)
        X[col] = pd.to_numeric(X[col], errors='coerce')

    return X, y


def fix_val_test_shape(X):
    lower_wns = 300
    upper_wns = 1942
    joint_wns = np.arange(lower_wns, upper_wns + 1)
    spectral_values = np.linspace(65, 3350, 2048)

    spectra_selection = np.logical_and(
        lower_wns <= spectral_values, spectral_values <= upper_wns,
    )
    wns = spectral_values[spectra_selection]
    X = X[:, spectra_selection]
    X = np.array([np.interp(joint_wns, xp=wns, fp=spectrum,)for spectrum in X])
    return X

In [6]:
inputs, targets = load_comp_data(os.path.join(path, 'transfer_plate.csv'), is_train=True)
test_inputs, _ = load_comp_data(os.path.join(path, '96_samples.csv'), is_train=False)

In [7]:
inputs = inputs.drop('sample_id', axis=1).values.reshape(-1, 2, 2048).mean(axis=1)
test_inputs = test_inputs.drop('sample_id', axis=1).values.reshape(-1, 2, 2048).mean(axis=1)

inputs = fix_val_test_shape(inputs)
test_inputs = fix_val_test_shape(test_inputs)

# Version 2 Update: Normalise Val and Test data like Train
inputs = inputs / np.max(inputs)
test_inputs = test_inputs / np.max(test_inputs)

In [8]:
inputs.shape, targets.shape, test_inputs.shape

((96, 1643), (96, 3), (96, 1643))

In [9]:
import numpy as np
import random
import torch
from torch.utils.data import Dataset
import scipy.optimize


np_dtype_from_torch = {
    torch.float32: np.float32,
    torch.float64: np.float64,
}

class SpectralDataset(Dataset):
    def __init__(
        self,
        spectra,
        concentrations,
        dtype=None,
        spectra_mean_std=None,
        concentration_mean_std=None,
        combine_spectra_range=0.0,
        baseline_factor_bound=0.0,
        baseline_period_lower_bound=100.0,
        baseline_period_upper_bound=200.0,
        augment_slope_std=0.0,
        augment_intersept_std=0.0,
        rolling_bound=0,
        spectrum_rolling_sigma=0.0,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    ):
        self.dtype = dtype or torch.float32
        self.combine_spectra_range = combine_spectra_range
        self.baseline_factor_bound = baseline_factor_bound
        self.augment_slope_std = augment_slope_std
        self.augment_intercept_std = augment_intersept_std
        self.baseline_period_lower_bound = baseline_period_lower_bound
        self.baseline_period_upper_bound = baseline_period_upper_bound
        self.rolling_bound = rolling_bound
        self.spectrum_rolling_sigma = spectrum_rolling_sigma
        self.augmentation_weight = torch.tensor(augmentation_weight, dtype=dtype)
        self.original_dp_weight = original_datapoint_weight

        # normalize spectra
        spectra = torch.tensor(spectra, dtype=dtype)

        if spectra_mean_std is None:
            self.s_mean = torch.mean(spectra)
            self.s_std = torch.std(spectra)
        else:
            self.s_mean, self.s_std = spectra_mean_std

        self.spectra = torch.divide(
            torch.subtract(spectra, self.s_mean),
            self.s_std,
        )

        self.dummy_wns = np.tile(
            np.arange(
                0., 1., 1. / self.spectra.shape[2],
                dtype=np_dtype_from_torch[self.dtype]
            )[None, :self.spectra.shape[2]],
            (self.spectra.shape[1], 1),
        )

        # normalize concentrations
        concentrations = torch.tensor(concentrations, dtype=dtype)
        if concentration_mean_std is None:
            self.concentration_means = torch.nanmean(concentrations, dim=0)

            self.concentration_stds = torch.maximum(
                torch.tensor(
                    [
                        torch.std(col[torch.logical_not(torch.isnan(col))])
                        for col in concentrations.T
                    ]
                ),
                torch.tensor([1e-3] * concentrations.shape[1]),
            )
        else:
            self.concentration_means = concentration_mean_std[0]
            self.concentration_stds = concentration_mean_std[1]

        self.concentrations = torch.divide(
            torch.subtract(
                concentrations,
                self.concentration_means,
            ),
            self.concentration_stds,
        )

    def pick_two(self, max_idx=None):
        max_idx = max_idx or len(self)
        return random.choices(range(max_idx), k=2)

    def __len__(self):
        return len(self.concentrations)

    def augment_spectra(self, spectra):
        if self.augment_slope_std > 0.0:

            def spectrum_approximation(x, slope, intercept):
                return (slope * x + intercept).reshape(-1, 1)[:, 0]

            slope, inter = scipy.optimize.curve_fit(
                spectrum_approximation,
                self.dummy_wns,
                spectra.reshape(-1, 1)[:, 0],
                p0=np.random.rand(2),
            )[0]

            new_slope = slope * (
                    np.random.gamma(
                        shape=1. / self.augment_slope_std,
                        scale=self.augment_slope_std,
                        size=1,
                    )
            )[0]
            new_intercept = inter * (
                1.0 + np.random.randn(1) * self.augment_intercept_std
            )[0]
            spectra += torch.tensor(
                (new_slope - slope)
            ) * self.dummy_wns + new_intercept - inter

        factor = self.baseline_factor_bound * torch.rand(size=(1,))
        offset = torch.rand(size=(1,)) * 2.0 * torch.pi
        period = self.baseline_period_lower_bound + (
            self.baseline_period_upper_bound - self.baseline_period_lower_bound
        ) * torch.rand(size=(1,))
        permutations = factor * torch.cos(
            2.0 * torch.pi / period * self.dummy_wns + offset
        )
        return self.roll_spectrum(
            spectra + permutations * spectra,
            delta=random.randint(-self.rolling_bound, self.rolling_bound),
        )

    def roll_spectrum(self, spectra, delta):
        num_spectra = spectra.shape[0]
        rolled_spectra = np.roll(spectra, delta, axis=1)
        if delta > 0:
            rolled_spectra[:, :delta] = (
                np.random.rand(num_spectra, delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta:(delta + 1)]
        elif delta < 0:
            rolled_spectra[:, delta:] = (
                np.random.rand(num_spectra, -delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta - 1:delta]
        return rolled_spectra

    def combine_k_items(self, indices, weights):
        return (
            # spectra
            torch.sum(
                torch.mul(weights[:, None, None], self.spectra[indices, :, :]),
                dim=0,
            ),
            # concentrations
            torch.sum(
                torch.mul(weights[:, None], self.concentrations[indices, :]),
                dim=0,
            )
        )

    def __getitem__(self, idx):
        if self.combine_spectra_range < 1e-12:
            spectrum = self.spectra[idx]
            spectrum = self.augment_spectra(spectrum)
            return {
                "spectra": spectrum,
                "concentrations": self.concentrations[idx],
                "label_weight": torch.tensor(1.0, dtype=self.dtype),
            }
        else:
            if random.random() < self.original_dp_weight:
                one_weight = 1.
                label_weight = torch.tensor(1.0, dtype=self.dtype)
            else:
                one_weight = random.uniform(0.0, self.combine_spectra_range)
                label_weight = self.augmentation_weight
            weights = torch.tensor([one_weight, (1 - one_weight)])
            # just pick two random indices
            indices = random.choices(range(len(self)), k=2)

            mixed_spectra, mixed_concentrations = self.combine_k_items(
                indices=indices,
                weights=weights,
            )
            mixed_spectra = self.augment_spectra(mixed_spectra)
            return mixed_spectra, mixed_concentrations, label_weight


config = {
    'initial_cnn_channels': 32,
    'cnn_channel_factor': 1.279574024454846,
    'num_cnn_layers': 8,
    'kernel_size': 3,
    'stride': 2,
    'activation_function': 'ELU',
    'fc_dropout': 0.10361700399831791,
    'lr': 0.001,
    'gamma': 0.9649606352621118,
    'baseline_factor_bound': 0.748262317340447,
    'baseline_period_lower_bound': 0.9703081695287203,
    'baseline_period_span': 19.79744237606427,
    'original_datapoint_weight': 0.4335003268130408,
    'augment_slope_std': 0.08171025264382692,
    'batch_size': 32,
    'fc_dims': 226,
    'rolling_bound': 2,
    'num_blocks': 2,
}

def get_dataset(inputs, targets, config, inputs_mean_std=None, targets_mean_std=None):
    return SpectralDataset(
        spectra=inputs[:, None, :],
        concentrations=targets,
        dtype=torch.float32,
        spectra_mean_std=inputs_mean_std,
        concentration_mean_std=targets_mean_std,
        combine_spectra_range=1.0,
        baseline_factor_bound=config["baseline_factor_bound"],
        baseline_period_lower_bound=config["baseline_period_lower_bound"],
        baseline_period_upper_bound=(config["baseline_period_lower_bound"] + config["baseline_period_span"]),
        augment_slope_std=config["augment_slope_std"],
        augment_intersept_std=0.0,
        rolling_bound=config["rolling_bound"],
        spectrum_rolling_sigma=0.01,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    )

In [10]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+5232)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
def return_dls(train_ds, eval_ds, train_batch_size, eval_batch_size):
    train_dl = build_loader(
        SEED,
        train_ds,
        train=True,
        batch_size=train_batch_size,
        shuffle=True,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )

    eval_dl = build_loader(
        SEED,
        eval_ds,
        train=False,
        batch_size=eval_batch_size,
        shuffle=False,
        num_workers=0,
        drop_last=False,
        pin_memory=True,
        persistent_workers=False,
    )
    
    return train_dl, eval_dl

In [11]:
import neptune


def setup_neptune():
    if not RESUME:
        neptune_run = neptune.init_run(
            project="arbaaz/kaggle-spect",
            name=MODEL_NAME,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

        neptune_run["h_parameters"] = {
            "seed": SEED,
            "model_name": MODEL_NAME,
            "optimizer_name": "nadam",
            "learning_rate": LR,
            "scheduler_name": "default",
            "weight_decay": WD,
            "num_epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
        }
        if DROPOUT: neptune_run["h_parameters"] = {"dropout": DROPOUT}
        if DROP_PATH_RATE: neptune_run["h_parameters"] = {"drop_path_rate": DROP_PATH_RATE}
    else:
        neptune_run = neptune.init_run(
            project="arbaaz/crunchdao-structural-break",
            with_id=config.with_id,
            api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
        )

    return neptune_run

In [12]:
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss
from sklearn.metrics import r2_score


def loss_fn(logits, targets):
    logits = logits.view(-1)
    targets = targets.view(-1)
    return F.mse_loss(logits, targets)


def metric_fn(logits, targets):
    preds = logits.cpu().detach().float().numpy()
    targets = targets.cpu().detach().float().numpy()
    
    dim1 = r2_score(targets[:, 0], preds[:, 0])
    dim2 = r2_score(targets[:, 1], preds[:, 1])
    dim3 = r2_score(targets[:, 2], preds[:, 2])
    
    return dim1, dim2, dim3, r2_score(targets, preds)


class MSEIgnoreNans(_Loss):
    def forward(
        self,
        input: torch.Tensor,
        target: torch.Tensor,
        weights: torch.Tensor,
    ) -> torch.Tensor:
        mask = torch.isfinite(target)
        mse = torch.mean(
            torch.mul(
                torch.square(input[mask] - target[mask]),
                torch.tile(weights[:, None], dims=(1, target.shape[1]))[mask],
            )
        )
        return torch.where(
            torch.isfinite(mse),
            mse,
            torch.tensor(0.).to(target.device),
        )

In [13]:
import math


class Identity(torch.torch.nn.Module):
    def forward(self, x):
        return x


# this is not a resnet yet
class ReZeroBlock(torch.torch.nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        activation_function,
        kernel_size,
        stride,
        dtype,
        norm_layer=None,
    ):
        super(ReZeroBlock, self).__init__()
        if norm_layer is None:
            norm_layer = torch.torch.nn.BatchNorm1d

        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = divmod(kernel_size, 2)[0] if stride == 1 else 0

        # does not change spatial dimension
        self.conv1 = torch.nn.Conv1d(
            in_channels,
            out_channels,
            kernel_size=1,
            stride=1,
            bias=False,
            dtype=dtype,
        )
        self.bn1 = norm_layer(out_channels, dtype=dtype)
        # Both self.conv2 and self.downsample layers
        # downsample the input when stride != 1
        self.conv2 = torch.nn.Conv1d(
            out_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            groups=out_channels,
            bias=False,
            dtype=dtype,
            padding=self.padding,
        )
        if stride > 1:
            down_conv = torch.nn.Conv1d(
                in_channels,
                out_channels,
                kernel_size=kernel_size,
                stride=stride,
                bias=False,
                dtype=dtype,
                # groups=out_channels,
            )
        else:
            down_conv = Identity()

        self.down_sample = torch.nn.Sequential(
            down_conv,
            norm_layer(out_channels),
        )
        self.bn2 = norm_layer(out_channels, dtype=dtype)
        # does not change the spatial dimension
        self.conv3 = torch.nn.Conv1d(
            out_channels,
            out_channels,
            kernel_size=1,
            stride=1,
            bias=False,
            dtype=dtype,
        )
        self.bn3 = norm_layer(out_channels, dtype=dtype)
        self.activation = activation_function(inplace=True)
        self.factor = torch.torch.nn.parameter.Parameter(torch.tensor(0.0, dtype=dtype))

    def next_spatial_dim(self, last_spatial_dim):
        return math.floor(
            (last_spatial_dim + 2 * self.padding - self.kernel_size)
            / self.stride + 1
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.activation(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.activation(out)

        out = self.conv3(out)
        out = self.bn3(out)

        # not really the identity, but kind of
        identity = self.down_sample(x)

        return self.activation(out * self.factor + identity)


class ResNetEncoder(torch.torch.nn.Module):
    def __init__(
        self,
        spectrum_size,
        cnn_encoder_channel_dims,
        activation_function,
        kernel_size,
        stride,
        dtype,
        num_blocks,
        verbose=False,
    ):
        super(ResNetEncoder, self).__init__()

        self.spatial_dims = [spectrum_size]
        layers = []
        for in_channels, out_channels in zip(
            cnn_encoder_channel_dims[:-1],
            cnn_encoder_channel_dims[1:],
        ):
            block = ReZeroBlock(
                in_channels=in_channels,
                out_channels=out_channels,
                activation_function=activation_function,
                kernel_size=kernel_size,
                stride=stride,
                dtype=dtype,
            )
            layers.append(block)
            self.spatial_dims.append(block.next_spatial_dim(self.spatial_dims[-1]))
            for _ in range(num_blocks - 1):
                block = ReZeroBlock(
                    in_channels=out_channels,
                    out_channels=out_channels,
                    activation_function=activation_function,
                    kernel_size=kernel_size,
                    stride=1,
                    dtype=dtype,
                )
                layers.append(block)
                self.spatial_dims.append(block.next_spatial_dim(self.spatial_dims[-1]))

        self.resnet_layers = torch.torch.nn.Sequential(*layers)
        if verbose:
            print("CNN Encoder Channel Dims: %s" % (cnn_encoder_channel_dims))
            print("CNN Encoder Spatial Dims: %s" % (self.spatial_dims))

    def forward(self, x):
        return self.resnet_layers(x)


class ReZeroNet(torch.nn.Module):
    def __init__(
        self,
        spectra_channels,
        spectra_size,
        initial_cnn_channels,
        cnn_channel_factor,
        num_cnn_layers,
        kernel_size,
        stride,
        activation_function,
        fc_dims,
        fc_dropout=0.0,
        dtype=None,
        verbose=False,
        fc_output_channels=1,
        num_blocks=1,
        **kwargs,
    ):
        super().__init__()
        self.fc_output_channels = fc_output_channels
        self.dtype = dtype or torch.float32

        activation_function = getattr(torch.nn, activation_function)

        # Setup CNN Encoder
        cnn_encoder_channel_dims = [spectra_channels] + [
            int(initial_cnn_channels * (cnn_channel_factor**idx))
            for idx in range(num_cnn_layers)
        ]
        self.cnn_encoder = ResNetEncoder(
            spectrum_size=spectra_size,
            cnn_encoder_channel_dims=cnn_encoder_channel_dims,
            activation_function=activation_function,
            kernel_size=kernel_size,
            stride=stride,
            num_blocks=num_blocks,
            dtype=dtype,
            verbose=verbose,
        )
        self.fc_dims = [
            int(
                self.cnn_encoder.spatial_dims[-1]
            ) * int(cnn_encoder_channel_dims[-1])
        ] + fc_dims

        if verbose:
            print("Fc Dims: %s" % self.fc_dims)
        fc_layers = []
        for idx, (in_dim, out_dim) in enumerate(
                zip(self.fc_dims[:-2], self.fc_dims[1:-1])
        ):
            fc_layers.append(torch.nn.Linear(in_dim, out_dim))
            fc_layers.append(torch.nn.ELU())
            fc_layers.append(torch.nn.Dropout(fc_dropout / (2 ** idx)))
        fc_layers.append(
            torch.nn.Linear(
                self.fc_dims[-2],
                self.fc_dims[-1] * self.fc_output_channels,
            ),
        )
        self.fc_net = torch.nn.Sequential(*fc_layers)
        if verbose:
            num_params = sum(p.numel() for p in self.parameters())
            print("Number of Parameters: %s" % num_params)

    def forward(self, spectra):
        embeddings = self.cnn_encoder(spectra)
        forecast = self.fc_net(embeddings.view(-1, self.fc_dims[0]))
        if self.fc_output_channels > 1:
            forecast = forecast.reshape(
                -1, self.fc_output_channels, self.fc_dims[-1]
            )
        return forecast


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class LayerNorm(nn.Module):
    """LayerNorm for 1D with channels_last (N, L, C) or channels_first (N, C, L)."""
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        if data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError
        self.data_format = data_format
        self.normalized_shape = (normalized_shape,)

    def forward(self, x):
        if self.data_format == "channels_last":
            # x: (N, L, C); normalize over last dim C
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        else:
            # x: (N, C, L); normalize over channel dim C, per position L
            u = x.mean(dim=1, keepdim=True)                       # (N,1,L)
            s = (x - u).pow(2).mean(dim=1, keepdim=True)          # (N,1,L)
            x = (x - u) / torch.sqrt(s + self.eps)                # (N,C,L)
            # Broadcast weight/bias over (C,1)
            x = self.weight[:, None] * x + self.bias[:, None]
            return x
        
   
class GRN(nn.Module):
    """GRN for 1D sequences in channels-last format (N, L, C)."""
    def __init__(self, dim, eps=1e-6):
        super().__init__()
        # Broadcast over (N, L, C): parameters shape to (1,1,C)
        self.gamma = nn.Parameter(torch.zeros(1, 1, dim))
        self.beta = nn.Parameter(torch.zeros(1, 1, dim))
        self.eps = eps

    def forward(self, x):
        # x: (N, L, C)
        # L2 norm across channels at each position
        Gx = torch.norm(x, p=2, dim=-1, keepdim=True)                 # (N, L, 1)
        # Normalize by mean over positions (sequence length)
        Nx = Gx / (Gx.mean(dim=1, keepdim=True) + self.eps)           # (N, L, 1)
        return self.gamma * (x * Nx) + self.beta + x                  # (N, L, C)
    
    
from timm.layers import trunc_normal_, DropPath


class Block(nn.Module):
    """ ConvNeXtV2 Block (1D).
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
    """
    def __init__(self, dim, drop_path=0.):
        super().__init__()
        self.dwconv = nn.Conv1d(dim, dim, kernel_size=7, padding=3, groups=dim)  # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim)  # pointwise convs via linear
        self.act = nn.GELU()
        self.grn = GRN(4 * dim)
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)                 # (N, C, L)
        x = x.transpose(1, 2)              # (N, L, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.grn(x)
        x = self.pwconv2(x)
        x = x.transpose(1, 2)              # (N, C, L)

        x = input + self.drop_path(x)
        return x
    
    
class ConvNeXtV2(nn.Module):
    """ ConvNeXt V2 (1D)

    Args:
        in_chans (int): Number of input channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=1, num_classes=3,
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768],
                 drop_path_rate=0., dropout=0.1, head_init_scale=1.
                 ):
        super().__init__()
        self.depths = depths
        self.downsample_layers = nn.ModuleList()  # stem + 3 intermediate downsampling conv layers

        # Stem: 1D patchify with stride 4
        stem = nn.Sequential(
            nn.Conv1d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first"),
        )
        self.downsample_layers.append(stem)

        # Three downsample layers: halve length each stage
        for i in range(3):
            downsample_layer = nn.Sequential(
                LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                nn.Conv1d(dims[i], dims[i + 1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        # Stages with residual Blocks (already 1D)
        self.stages = nn.ModuleList()
        dp_rates = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j]) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        # Final norm and head: pool over length, then LayerNorm over channels
        self.norm = nn.LayerNorm(dims[-1], eps=1e-6)
        self.dropout = nn.Dropout(dropout)
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv1d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        # x: (N, C_in, L)
        for i in range(4):
            x = self.downsample_layers[i](x)  # (N, C_i, L_i)
            x = self.stages[i](x)             # (N, C_i, L_i)

        # Global average pooling over length -> (N, C)
        x = x.mean(dim=-1)

        # Final LayerNorm expects (N, C)
        x = self.norm(x)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.dropout(x)
        x = self.head(x)
        return x
    
    
    
    #def convnext_pico(**kwargs):
#    model = ConvNeXtV2(depths=[2, 2, 6, 2], dims=[64, 128, 256, 512], **kwargs)
#    return model

def convnextv2_atto(**kwargs):
    model = ConvNeXtV2(depths=[2, 2, 6, 2], dims=[40, 80, 160, 320], **kwargs)
    return model

In [15]:
from tqdm.auto import tqdm


def train(
    model, 
    optimizer,
    device,
    amp_dtype,
    scheduler,
    train_dl,
    eval_dl,
    loss_fn,
    epochs,
    checkpoint_name,
    score=-float("inf"),
    neptune_run=None,
    p=True,
):  
    scaler = torch.amp.GradScaler(device)
    for epoch in tqdm(range(epochs)):
        model.train()
        total_loss = 0.0
        all_logits = []
        all_targets = []
        
        for inputs, targets, weights in train_dl:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            weights = weights.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            with torch.amp.autocast(device_type=device, dtype=amp_dtype, cache_enabled=True):
                logits = model(inputs)
                loss = loss_fn(logits, targets)
                  
            if amp_dtype == torch.bfloat16:                
                loss.backward()
                optimizer.step()
            else:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()

            scheduler.step()
            if neptune_run is not None:  neptune_run["lr_step"].append(scheduler.get_last_lr()[0])
            
            total_loss += loss.detach().cpu()
            all_logits.append(logits.detach().cpu())
            all_targets.append(targets.detach().cpu())
        
        all_logits = torch.cat(all_logits)
        all_targets = torch.cat(all_targets)

        one, two, three, r2 = metric_fn(all_logits, all_targets)
        total_loss = total_loss / len(train_dl)
        
        model.eval()
        eval_total_loss = 0.0
        eval_all_logits = []
        eval_all_targets = []

        for inputs, targets, weights in eval_dl:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            weights = weights.to(device, non_blocking=True)

            with torch.inference_mode():
                with torch.amp.autocast(device_type=device, dtype=amp_dtype, cache_enabled=True):
                    logits = model(inputs)
                    loss = loss_fn(logits, targets)

            eval_total_loss += loss.detach().cpu()
            eval_all_logits.append(logits.detach().cpu())
            eval_all_targets.append(targets.detach().cpu())
        
        eval_all_logits = torch.cat(eval_all_logits)
        eval_all_targets = torch.cat(eval_all_targets)

        eval_one, eval_two, eval_three, eval_r2 = metric_fn(eval_all_logits, eval_all_targets)
        eval_total_loss = eval_total_loss / len(eval_dl)
        
        if eval_r2 > score:
            score = eval_r2
            data = {"state_dict": model.state_dict()}
            data["epoch"] = epoch 
            data["score"] = score
            torch.save(data, f"/kaggle/working/{checkpoint_name}")
        
        if neptune_run is not None:
            neptune_run["train/loss"].append(total_loss)
            neptune_run["eval/loss"].append(eval_total_loss)
            neptune_run["train/r2"].append(r2)
            neptune_run["eval/r2"].append(eval_r2)
            neptune_run["train/one"].append(one)
            neptune_run["train/two"].append(two)
            neptune_run["train/three"].append(three)
            neptune_run["eval/one"].append(eval_one)
            neptune_run["eval/two"].append(eval_two)
            neptune_run["eval/three"].append(eval_three)
            
        if p and epoch % 5 == 0:
            print(
                f"Epoch: {epoch}, "
                f"train/loss: {total_loss:.4f}, "
                f"eval/loss: {eval_total_loss:.4f}, "
                f"train/r2: {r2:.4f}, "
                f"eval/r2: {eval_r2:.4f}, "
                f"train/one: {one:.4f}, "
                f"train/two: {two:.4f}, "
                f"train/three: {three:.4f}, "
                f"eval/one: {eval_one:.4f}, "
                f"eval/two: {eval_two:.4f}, "
                f"eval/three: {eval_three:.4f} "
            )
            
    if neptune_run is not None: neptune_run.stop()
    return score

In [16]:
EPOCHS = 100
WD = 1e-3
LR = 1e-4

DROPOUT = 0.5
DROP_PATH_RATE = None

device = "cuda" if torch.cuda.is_available() else "cpu"
RESUME = False

config["dtype"] = torch.float32
config["spectra_size"] = 1643
config["spectra_channels"] = 1
config["fc_dims"] = [
    config["fc_dims"],
    int(config["fc_dims"] / 2),
    3,
]

mse_loss_function = MSEIgnoreNans()

In [17]:
from sklearn.model_selection import KFold


inputs_mean_std = []
targets_mean_std = []
scores = []
kfold = KFold(n_splits=5, shuffle=True, random_state=SEED)
splits = kfold.split(inputs)

for fold, (train_idx, eval_idx) in enumerate(splits):
    MODEL_NAME = f"resnet.finetune.avg.weights.p.data.fixed.F{fold}"
    checkpoint_name = f"resnet.finetune.avg.weights.p.ckpt.data.fixed.F{fold}.pt"
    
    train_inputs = inputs[train_idx]
    train_targets = targets[train_idx]
    eval_inputs = inputs[eval_idx]
    eval_targets = targets[eval_idx]

    train_ds = get_dataset(train_inputs, train_targets, config)
    
    inputs_mean_std.append((fold, train_ds.s_mean, train_ds.s_std))
    targets_mean_std.append((fold, train_ds.concentration_means, train_ds.concentration_stds))
    
    eval_ds = get_dataset(
        eval_inputs, 
        eval_targets, 
        config,
        (train_ds.s_mean, train_ds.s_std), 
        (train_ds.concentration_means, train_ds.concentration_stds)
    )
    
    BATCH_SIZE = 32
    train_dl, eval_dl = return_dls(train_ds, eval_ds, BATCH_SIZE, len(eval_ds))
    
    #model = convnextv2_atto().to(device)
    model = ReZeroNet(**config).to(device)
    ckpt = get_ckpt("/kaggle/working/avg_weights_data_fixed.pt")#["state_dict"]
    model.load_state_dict(ckpt)
    
    if fold == 0: print(get_model_size(model))
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD, foreach=True)
    scheduler = get_scheduler(optimizer, train_dl, EPOCHS)
    
    score = train(
            model, 
            optimizer, 
            device,
            torch.float16,
            scheduler,
            train_dl, 
            eval_dl,
            loss_fn,
            EPOCHS,
            checkpoint_name,
            neptune_run=setup_neptune(),
        )
    
    scores.append(score)

0.734309
None


[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs-legacy.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/arbaaz/kaggle-spect/e/KAG-271


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0, train/loss: 1.0304, eval/loss: 0.8932, train/r2: -0.0086, eval/r2: -0.1388, train/one: -0.0038, train/two: -0.0260, train/three: 0.0040, eval/one: -0.0053, eval/two: -0.0083, eval/three: -0.4029 
Epoch: 5, train/loss: 0.9350, eval/loss: 1.0406, train/r2: -0.0284, eval/r2: -0.1738, train/one: -0.0783, train/two: -0.0111, train/three: 0.0042, eval/one: -0.3386, eval/two: -0.1173, eval/three: -0.0653 
Epoch: 10, train/loss: 0.8460, eval/loss: 0.9769, train/r2: 0.0862, eval/r2: -0.1147, train/one: -0.0091, train/two: 0.0030, train/three: 0.2647, eval/one: 0.0020, eval/two: -0.2995, eval/three: -0.0464 
Epoch: 15, train/loss: 0.7863, eval/loss: 0.9368, train/r2: 0.2077, eval/r2: 0.0252, train/one: -0.0025, train/two: -0.0413, train/three: 0.6668, eval/one: -0.0930, eval/two: -0.0519, eval/three: 0.2203 
Epoch: 20, train/loss: 0.7106, eval/loss: 0.8992, train/r2: 0.2487, eval/r2: -0.0785, train/one: -0.0533, train/two: -0.0302, train/three: 0.8296, eval/one: 0.0157, eval/two: -0.09

...

Epoch: 0, train/loss: 0.9650, eval/loss: 1.3911, train/r2: -0.0220, eval/r2: -0.0207, train/one: -0.0246, train/two: -0.0036, train/three: -0.0379, eval/one: -0.0274, eval/two: -0.0290, eval/three: -0.0056 
Epoch: 5, train/loss: 0.9236, eval/loss: 0.9266, train/r2: 0.0051, eval/r2: -0.1875, train/one: -0.0278, train/two: -0.0024, train/three: 0.0455, eval/one: -0.2048, eval/two: -0.1039, eval/three: -0.2538 
Epoch: 10, train/loss: 0.8926, eval/loss: 1.1786, train/r2: 0.0646, eval/r2: -0.0906, train/one: -0.0333, train/two: 0.0049, train/three: 0.2223, eval/one: -0.0090, eval/two: -0.2742, eval/three: 0.0115 
Epoch: 15, train/loss: 0.7762, eval/loss: 0.9625, train/r2: 0.2232, eval/r2: -0.1435, train/one: 0.0020, train/two: 0.0063, train/three: 0.6613, eval/one: -0.0572, eval/two: -0.2940, eval/three: -0.0794 
Epoch: 20, train/loss: 0.7193, eval/loss: 0.8702, train/r2: 0.3006, eval/r2: 0.0947, train/one: 0.0373, train/two: 0.0230, train/three: 0.8413, eval/one: 0.0544, eval/two: -0.3327,

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0, train/loss: 1.0381, eval/loss: 1.5364, train/r2: -0.0151, eval/r2: -0.1736, train/one: -0.0028, train/two: -0.0063, train/three: -0.0363, eval/one: -0.2290, eval/two: -0.0016, eval/three: -0.2902 
Epoch: 5, train/loss: 0.8713, eval/loss: 1.2600, train/r2: -0.0318, eval/r2: -0.4653, train/one: -0.0243, train/two: -0.0209, train/three: -0.0502, eval/one: -0.0745, eval/two: -0.4591, eval/three: -0.8623 
Epoch: 10, train/loss: 0.9980, eval/loss: 1.4902, train/r2: 0.0651, eval/r2: -0.4979, train/one: -0.0204, train/two: 0.0083, train/three: 0.2074, eval/one: -1.3945, eval/two: -0.0637, eval/three: -0.0357 
Epoch: 15, train/loss: 0.7737, eval/loss: 1.2890, train/r2: 0.2103, eval/r2: 0.0350, train/one: -0.0006, train/two: 0.0101, train/three: 0.6215, eval/one: -0.0381, eval/two: -0.0075, eval/three: 0.1507 
Epoch: 20, train/loss: 0.6772, eval/loss: 0.8765, train/r2: 0.2598, eval/r2: 0.1152, train/one: 0.0140, train/two: 0.0008, train/three: 0.7644, eval/one: -0.1376, eval/two: -0.05

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0, train/loss: 1.0525, eval/loss: 0.9772, train/r2: -0.0045, eval/r2: -0.0379, train/one: -0.0030, train/two: -0.0014, train/three: -0.0090, eval/one: -0.0017, eval/two: -0.0325, eval/three: -0.0796 
Epoch: 5, train/loss: 0.9978, eval/loss: 0.9386, train/r2: 0.0080, eval/r2: -0.2470, train/one: -0.0003, train/two: -0.0238, train/three: 0.0482, eval/one: -0.0100, eval/two: -0.3368, eval/three: -0.3941 
Epoch: 10, train/loss: 0.9016, eval/loss: 0.8882, train/r2: 0.0949, eval/r2: -0.0844, train/one: -0.0507, train/two: 0.0290, train/three: 0.3064, eval/one: -0.0015, eval/two: -0.0334, eval/three: -0.2184 
Epoch: 15, train/loss: 0.7366, eval/loss: 0.7502, train/r2: 0.2829, eval/r2: -0.0418, train/one: 0.0396, train/two: 0.0861, train/three: 0.7231, eval/one: -0.1079, eval/two: -0.1962, eval/three: 0.1786 
Epoch: 20, train/loss: 0.7314, eval/loss: 0.8153, train/r2: 0.2908, eval/r2: -0.0213, train/one: 0.0631, train/two: 0.0032, train/three: 0.8060, eval/one: -0.0921, eval/two: -0.210

In [18]:
class SpectralTestDataset(Dataset):
    def __init__(
        self,
        spectra,
        concentrations,
        dtype=None,
        spectra_mean_std=None,
        concentration_mean_std=None,
        combine_spectra_range=0.0,
        baseline_factor_bound=0.0,
        baseline_period_lower_bound=100.0,
        baseline_period_upper_bound=200.0,
        augment_slope_std=0.0,
        augment_intersept_std=0.0,
        rolling_bound=0,
        spectrum_rolling_sigma=0.0,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    ):
        self.dtype = dtype or torch.float32
        self.combine_spectra_range = combine_spectra_range
        self.baseline_factor_bound = baseline_factor_bound
        self.augment_slope_std = augment_slope_std
        self.augment_intercept_std = augment_intersept_std
        self.baseline_period_lower_bound = baseline_period_lower_bound
        self.baseline_period_upper_bound = baseline_period_upper_bound
        self.rolling_bound = rolling_bound
        self.spectrum_rolling_sigma = spectrum_rolling_sigma
        self.augmentation_weight = torch.tensor(augmentation_weight, dtype=dtype)
        self.original_dp_weight = original_datapoint_weight

        # normalize spectra
        spectra = torch.tensor(spectra, dtype=dtype)

        if spectra_mean_std is None:
            self.s_mean = torch.mean(spectra)
            self.s_std = torch.std(spectra)
        else:
            self.s_mean, self.s_std = spectra_mean_std

        self.spectra = torch.divide(
            torch.subtract(spectra, self.s_mean),
            self.s_std,
        )

        self.dummy_wns = np.tile(
            np.arange(
                0., 1., 1. / self.spectra.shape[2],
                dtype=np_dtype_from_torch[self.dtype]
            )[None, :self.spectra.shape[2]],
            (self.spectra.shape[1], 1),
        )

        if False:
            # normalize concentrations
            concentrations = torch.tensor(concentrations, dtype=dtype)
            if concentration_mean_std is None:
                self.concentration_means = torch.nanmean(concentrations, dim=0)

                self.concentration_stds = torch.maximum(
                    torch.tensor(
                        [
                            torch.std(col[torch.logical_not(torch.isnan(col))])
                            for col in concentrations.T
                        ]
                    ),
                    torch.tensor([1e-3] * concentrations.shape[1]),
                )
            else:
                self.concentration_means = concentration_mean_std[0]
                self.concentration_stds = concentration_mean_std[1]

            self.concentrations = torch.divide(
                torch.subtract(
                    concentrations,
                    self.concentration_means,
                ),
                self.concentration_stds,
            )

    def pick_two(self, max_idx=None):
        max_idx = max_idx or len(self)
        return random.choices(range(max_idx), k=2)

    def __len__(self):
        return 96

    def augment_spectra(self, spectra):
        if self.augment_slope_std > 0.0:

            def spectrum_approximation(x, slope, intercept):
                return (slope * x + intercept).reshape(-1, 1)[:, 0]

            slope, inter = scipy.optimize.curve_fit(
                spectrum_approximation,
                self.dummy_wns,
                spectra.reshape(-1, 1)[:, 0],
                p0=np.random.rand(2),
            )[0]

            new_slope = slope * (
                    np.random.gamma(
                        shape=1. / self.augment_slope_std,
                        scale=self.augment_slope_std,
                        size=1,
                    )
            )[0]
            new_intercept = inter * (
                1.0 + np.random.randn(1) * self.augment_intercept_std
            )[0]
            spectra += torch.tensor(
                (new_slope - slope)
            ) * self.dummy_wns + new_intercept - inter

        factor = self.baseline_factor_bound * torch.rand(size=(1,))
        offset = torch.rand(size=(1,)) * 2.0 * torch.pi
        period = self.baseline_period_lower_bound + (
            self.baseline_period_upper_bound - self.baseline_period_lower_bound
        ) * torch.rand(size=(1,))
        permutations = factor * torch.cos(
            2.0 * torch.pi / period * self.dummy_wns + offset
        )
        return self.roll_spectrum(
            spectra + permutations * spectra,
            delta=random.randint(-self.rolling_bound, self.rolling_bound),
        )

    def roll_spectrum(self, spectra, delta):
        num_spectra = spectra.shape[0]
        rolled_spectra = np.roll(spectra, delta, axis=1)
        if delta > 0:
            rolled_spectra[:, :delta] = (
                np.random.rand(num_spectra, delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta:(delta + 1)]
        elif delta < 0:
            rolled_spectra[:, delta:] = (
                np.random.rand(num_spectra, -delta) * self.spectrum_rolling_sigma + 1
            ) * rolled_spectra[:, delta - 1:delta]
        return rolled_spectra

    def combine_k_items(self, indices, weights):
        return (
            # spectra
            torch.sum(
                torch.mul(weights[:, None, None], self.spectra[indices, :, :]),
                dim=0,
            ),
            # concentrations
            #torch.sum(
            #    torch.mul(weights[:, None], self.concentrations[indices, :]),
            #    dim=0,
            #)
        )

    def __getitem__(self, idx):
        if True:#self.combine_spectra_range < 1e-12:
            spectrum = self.spectra[idx]
            #spectrum = self.augment_spectra(spectrum)
            return spectrum
        else:
            if random.random() < self.original_dp_weight:
                one_weight = 1.
                label_weight = torch.tensor(1.0, dtype=self.dtype)
            else:
                one_weight = random.uniform(0.0, self.combine_spectra_range)
                label_weight = self.augmentation_weight
            weights = torch.tensor([one_weight, (1 - one_weight)])
            # just pick two random indices
            indices = random.choices(range(len(self)), k=2)

            mixed_spectra = self.combine_k_items(
                indices=indices,
                weights=weights,
            )
            mixed_spectra = self.augment_spectra(mixed_spectra[0])
            return mixed_spectra
        
  
def get_test_dataset(inputs, inputs_mean_std, targets_mean_std):
    return SpectralTestDataset(
        spectra=inputs[:, None, :],
        concentrations=None,
        dtype=torch.float32,
        spectra_mean_std=inputs_mean_std,
        concentration_mean_std=targets_mean_std,
        combine_spectra_range=1.0,
        baseline_factor_bound=config["baseline_factor_bound"],
        baseline_period_lower_bound=config["baseline_period_lower_bound"],
        baseline_period_upper_bound=(config["baseline_period_lower_bound"] + config["baseline_period_span"]),
        augment_slope_std=config["augment_slope_std"],
        augment_intersept_std=0.0,
        rolling_bound=config["rolling_bound"],
        spectrum_rolling_sigma=0.01,
        augmentation_weight=0.1,
        original_datapoint_weight=1.,
    )

In [22]:
ckpt_paths = get_ckpt_paths("/kaggle/working/", "avg.weights.p")

/kaggle/working/resnet.finetune.avg.weights.p.ckpt.data.fixed.F0.pt 73 0.890884074150463
/kaggle/working/resnet.finetune.avg.weights.p.ckpt.data.fixed.F1.pt 83 0.9202159693697608
/kaggle/working/resnet.finetune.avg.weights.p.ckpt.data.fixed.F2.pt 75 0.9141962849557639
/kaggle/working/resnet.finetune.avg.weights.p.ckpt.data.fixed.F3.pt 94 0.9173730713123304
/kaggle/working/resnet.finetune.avg.weights.p.ckpt.data.fixed.F4.pt 92 0.933128269953417


In [22]:
targets_mean_std[2][1:]

(tensor([6.9619, 1.1908, 1.5928]), tensor([2.8689, 0.5668, 0.6464]))

In [23]:
from torch.utils.data import DataLoader


def inference(test_inputs, ckpt_name, i):
    ckpt = get_ckpt(ckpt_name)
    
    test_ds = get_test_dataset(test_inputs, inputs_mean_std[i][1:], targets_mean_std[i][1:]) #[i][1:]
    test_dl = DataLoader(test_ds, batch_size=32)

    
    model = ReZeroNet(**config).to(device)
    model.load_state_dict(ckpt["state_dict"])
    model.eval()
    
    all_preds = []
    for inputs in test_dl:
        with torch.inference_mode():
            preds = model(inputs.cuda())
            preds = preds.double() 
            all_preds.append(cuda_to_np(preds))
            
    preds = np.concatenate(all_preds)
    mus = targets_mean_std[i][1:][0] #[i][1:][0]
    sigmas = targets_mean_std[i][1:][1] #[i][1:][1]

    for i in range(3):
        preds[:, i] = reverse_zscore(preds[:, i], mus[i].numpy(), sigmas[i].numpy())
    
    return preds

preds = inference(test_inputs, "/kaggle/working/resnet.finetune.avg.weights.p.ckpt.data.fixed.F4.pt", 4) # CAREFUL ABOUT INDEX
generate_csv(preds, "resnet.finetune.avg.weights.p.ckpt.data.fixed.F4.9331.csv")
preds

array([[2.35193701, 0.70662384, 0.61738674],
       [6.49085247, 1.96047767, 1.46188283],
       [4.19197596, 0.30039638, 1.03181022],
       [2.65833732, 0.77751213, 0.39134215],
       [9.63981799, 0.51798429, 0.97882175],
       [7.63364555, 1.93437445, 1.04053863],
       [5.38055328, 0.48491612, 0.32809626],
       [6.77800095, 1.96460996, 1.23086177],
       [7.15960728, 1.74662916, 1.0126448 ],
       [9.94141673, 0.70411267, 0.23760362],
       [8.93824441, 1.03869283, 0.97580407],
       [2.57959917, 0.79118592, 0.94668337],
       [2.7824985 , 1.18457097, 1.11126463],
       [3.64108348, 0.99573297, 1.37807992],
       [3.97725705, 0.87835026, 0.92747408],
       [7.05234218, 1.3170997 , 0.86020819],
       [3.24437878, 0.98439018, 1.03394707],
       [4.34187367, 0.99896576, 1.14726068],
       [4.68313124, 1.44564602, 1.1899677 ],
       [3.05547406, 0.90933227, 1.16126185],
       [3.49208235, 1.05638682, 1.04363395],
       [5.06547481, 1.05295697, 1.07452596],
       [2.

In [36]:
get_stats(targets, minmax=True), get_stats(preds, minmax=True)

Min: 0.276526487, Max: 11.88990894, Mean: 3.208722795402778, Std: 3.1291512817697695
Min: 0.03802425901420747, Max: 9.493976433275805, Mean: 2.430948129346428, Std: 2.314666198874381


(None, None)

In [24]:
get_stats(targets, minmax=True), get_stats(preds, minmax=True)

Min: 0.276526487, Max: 11.88990894, Mean: 3.208722795402778, Std: 3.1291512817697695
Min: 0.21520479832992123, Max: 9.941416729453636, Mean: 2.220094471803795, Std: 1.9768473694491508


(None, None)

In [ ]:
def ensemble_inference(ckpt_paths):
    test_inputs = get_test_data()
    all_preds = []

    for i, ckpt_path in enumerate(ckpt_paths):
        ckpt = get_ckpt(ckpt_path)
        
        model = ReZeroNet(**config).to(device)
        model.load_state_dict(ckpt["state_dict"])
        model.eval()

        test_ds = get_test_dataset(test_inputs, inputs_mean_std[i][1:], targets_mean_std[i][1:])
        test_dl = DataLoader(test_ds, batch_size=32)
        
        fold_preds = []
        for inputs in test_dl:
            with torch.inference_mode():
                preds = model(inputs.cuda())
                preds = cuda_to_np(preds.double())
                fold_preds.append(preds)
                
        fold_preds = np.concatenate(fold_preds)
        
        means = targets_mean_std[i][1:][0]
        stds = targets_mean_std[i][1:][1]
        for i in range(3):
            fold_preds[:, i] = reverse_zscore(fold_preds[:, i], means[i].numpy(), stds[i].numpy())
            
        all_preds.append(fold_preds)

    return np.mean(all_preds, axis=0)

preds = ensemble_inference(ckpt_paths)
generate_csv(preds, "paper.finetune.avg.pretrain.weights.ensemble.csv")
preds